#  <h2><center>PyVedaloss</center></h2>
   <h2><center>✵ Edition Navitas ✵</center></h2>
    <center>par Valente Antonin</center>
    </br>
   <center><img src="./logo.png" width="300"></center><h4>
   <center><img src="./Bandeau_logos_tutelles.png" width="300"></center><h4>

## Importation des modules

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error
from typing import Tuple


In [2]:
data_dataset = pd.read_csv('HBG_range_2.5_to_500_mev_per_nuc_1990_in_silicon_helium_to_calcium.csv')#.groupby(['Z','A'])
data_dataset = data_dataset.groupby(['A','Z','Z_material','A_material'])
#params_dataset = pd.read_csv('pyvedaloss_data.csv')
#Drop the columns that are not needed
# params_dataset = params_dataset.drop(['parameter_7',
#                                       'parameter_8',
#                                       'parameter_9',
#                                       'parameter_10',
#                                       'parameter_11',
#                                       'parameter_12',], axis=1)
# params_dataset = params_dataset.groupby('material').get_group('Silicium').groupby(['Z','A'])
# test = data_dataset.get_group((2, 4)) # = params_dataset.drop(['material'], axis=1)

In [3]:


def range_fit_closure(params: pd.DataFrame):
    """
    Relation de fermerture pour le fit de la range

    Paramètres
    ----------
    params : pd.DataFrame
        Dataframe contenant les paramètres du fit de la range
    """
    # Initilanise une variable avec les colonnes 'parameter_1' à 'parameter_5'
    params = params.loc[:, 'parameter_1':'parameter_6'].values[0]

    @np.vectorize
    def range_fit(x: float):
        """
        Calcule du fit de la courbe HBM

        Paramètres
        ----------
        x : float
            Energie par nucléon en MeV

        Retourne
        --------
        fit_values : float
        """
        fit_values = params[0] + params[1]*x + params[2] * \
            x**2 + params[3]*x**3 + params[4]*x**4  + \
        params[5]*x**5
        return fit_values

    return range_fit


def range_calculation_closure(range_fit: np.vectorize):
    """
    Relation de fermerture pour le calcul de la range

    Paramètres
    ----------
    range_fit : np.vectorize
        Fonction de fit de la range

    Retourne
    --------
    range_calculation : np.vectorize
    """

    @np.vectorize
    def range_calculation(x: float):
        """
        Calcule de la range en fonction de l'énergie par nucléon

        Paramètres
        ----------
        x : float
            Energie par nucléon en MeV
        """
        # Calcul des bases
        base_power_law_1 = np.log(0.1)
        base_power_law_2 = np.log(0.2)

        # Condition pour l'énergie inférieure à 0.1 MeV
        if x < 0.1:

            # Calcul de la range pour l'énergie inférieure à 0.1 MeV
            range_value_1 = range_fit(base_power_law_1)
            range_value_2 = range_fit(base_power_law_2)

            # Calcul de la dérivée ordre 1
            slope = range_value_2 - range_value_1
            slope /= base_power_law_2 - base_power_law_1

            # Développement de Taylor
            intercept = range_value_1 - slope * base_power_law_1

            # Valeurs du parcours
            range_value = slope * np.log(x) + intercept
        else:
            # Valeurs du parcours
            range_value = range_fit(np.log(x))

        # Ajout du terme log(A pure/A_silicium)
        range_value += np.log(28/28.0855)

        # Exponentielle
        range_value = np.exp(range_value)

        # Conversion en micron
        range_value /= 0.233
        return range_value
    return range_calculation


@staticmethod
def model(x, a, b):
    return np.power(x*a, b)


def benchmark(key: Tuple[int, int]) -> pd.DataFrame :
    """
    Fonction de benchmark pour le fit de la range

    Paramètres
    ----------
    key : Tuple[int, int]
        Tuple contenant le numéro atomique et le nombre de masse    
    
    Retourne
    --------
    pd.DataFrame : Dataframe contenant les données expérimentales et le fit
    """
    experiental_data = data_dataset.get_group(key)
    #fit_params = params_dataset.get_group(key)

    # Functions initialisation
    #range_fit = range_fit_closure(fit_params)
    #range_calculation = range_calculation_closure(range_fit)

    #Fit the range model
    popt, pcov = curve_fit(model, experiental_data.energy_per_nucleon_mev,
                           experiental_data.range_micron,
                           maxfev=1_000_000,
                           )
    # Calculate the R2 score
    r2 = r2_score(experiental_data.range_micron, model(
       experiental_data.energy_per_nucleon_mev, *popt))

    # Calculate the MSE
    mse = mean_squared_error(experiental_data.range_micron, model(
        experiental_data.energy_per_nucleon_mev, *popt))    
    
    # Calculate the RMSE
    rmse = np.sqrt(mse)

    # Calculate the MAPE
    mape = mean_absolute_percentage_error(experiental_data.range_micron, model(
        experiental_data.energy_per_nucleon_mev, *popt))

    # Calculate the R2 score of fit
    #r2_fit = 1-r2_score(experiental_data['Range (micron)'], range_calculation(
    #    experiental_data['Energy_per_nucleon (MeV)']))

    # Build figure for the benchmark
    fig = px.scatter(experiental_data, x='energy_per_nucleon_mev',
                     y='range_micron', title=f'Z={key[1]} A={key[0]}')
    energy_space = np.linspace(experiental_data.energy_per_nucleon_mev.min(
    ), experiental_data.energy_per_nucleon_mev.max(), 100)
    #fig.add_trace(px.line(x=energy_space,
    #                      y=range_calculation(energy_space),energy_per_nucleon_mev,range_micron
    #                      ).data[0])
    fig.add_trace(px.line(x=energy_space,
                          y=model(energy_space, *popt),
                          ).data[0])
    fig.update_layout(height=600,
                      width=800,
                      legend=dict(
                          yanchor="top",
                          y=0.99,
                          xanchor="left",
                          x=0.01
                      ))
    # Build relative residual figure for the benchmark
    #fig1 = px.scatter(x=experiental_data['Energy_per_nucleon (MeV)'],
    #                  y=(experiental_data['Range (micron)'] - range_calculation(
    #                     experiental_data['Energy_per_nucleon (MeV)']))/experiental_data['Range (micron)'],
    #                  title=f'Z={key[0]} A={key[1]}<br>Polynomial fit residual R2={r2}')
    # fig1.add_scatter(x=experiental_data['Energy_per_nucleon (MeV)'],
    #                  y=(experiental_data['Range (micron)'] - model(experiental_data['Energy_per_nucleon (MeV)'], *popt))/experiental_data['Range (micron)'], name='Power law fit residual')
    # fig1.update_layout(height=600,
    #                    width=800,
    #                    legend=dict(
    #                        yanchor="top",
    #                        y=0.99,
    #                        xanchor="left",
    #                        x=0.01
    #                    ))
    
    # Build the dataframe
    data = pd.DataFrame()
    data['Z_ion'] = [key[1]] 
    data['A_ion'] = [key[0]]
    data['Z_target'] = [key[2]]
    data['A_target'] = [key[3]]
    data['a'] = [popt[0]]
    data['b'] = [popt[1]]

    data['R2_power_law'] = r2
    data['MSE_power_law'] = mse
    data['RMSE_power_law'] = rmse
    data['MAPE_power_law'] = mape
    #data['R2_polynomial'] = r2_fit
    data['figure'] = [fig]
    #data['figure_residual'] = [fig1]
    #data['best_power_law'] = r2 < r2_fit
    return data

In [4]:
#Get keys of the dataset
keys = iter(data_dataset.groups.keys())

benchmark_dataset = pd.DataFrame()

while True:
    try :
        #Get the data and the parameters
        key = next(keys)
        benchmark_dataset = pd.concat([benchmark_dataset,benchmark(key)],axis=0)
    except StopIteration:
        break
benchmark_dataset.to_csv('Range_energy.csv',index=False, columns=['Z_ion','A_ion','Z_target','A_target','a','b'])

